# Day4 — Multisensor Forecast (Prophet regressors) + Compare (uni vs multi)
목표:
- Day3에서 확인한 단변량 토크 예측 한계를 바탕으로
- 토크(FB_Torque) 예측에 RPM(FB_Rpm)과 압력(Press_mean)을 회귀변수로 추가해 다변량 예측 수행
- 동일 레짐 구간(RPM>0, Torque<-50)에서 uni vs multi 성능(MAE/RMSE) 비교
- 예측 그래프 + 잔차(|actual-pred|) 비교 그래프 생성(figures/Day4)
- 지표 저장(tables/Day4/day4_metrics.csv)


## 입력/출력
- 입력(processed): data/processed/processed_v1_run_abs_features.csv
- 출력(표): tables/Day4/ (day4_metrics.csv, day4_regressor_standardization.csv)
- 출력(그림): figures/Day4/ (forecast_FB_Torque_compare.png)
- 실행 스크립트: src/day4_forecast_multisensor.py


## 1) 데이터 로드 & 기본 확인 (컬럼/스케일 sanity check)


In [ ]:
from pathlib import Path
import pandas as pd

PROCESSED = Path("../../data/processed/processed_v1_run_abs_features.csv")
TABLES_DIR = Path("../../tables/Day4")
FIG_DIR = Path("../../figures/Day4")

PROCESSED.exists(), TABLES_DIR.exists(), FIG_DIR.exists()

In [ ]:
df = pd.read_csv(PROCESSED)

df.shape, df.columns[:20]

## 2) Day4 산출물 생성 (스크립트 실행)
- Prophet 단변량(uni): y=FB_Torque
- Prophet 다변량(multi): y=FB_Torque + regressors=[FB_Rpm, Press_mean]
- resample=1s, run 기준 train/test split
- 평가 구간 레짐 필터: RPM>0 & Torque<-50
- 회귀변수는 train 통계로 표준화(z-score) 후 clipping 적용


In [ ]:
import subprocess
import sys

def run(cmd: str):
    print(">", cmd)
    res = subprocess.run(cmd, shell=True, text=True, capture_output=True)
    print(res.stdout)
    if res.returncode != 0:
        print(res.stderr)
    return res.returncode

py = sys.executable
run(f"{py} src/day4_forecast_multisensor.py")

## 3) 생성 결과 확인 (파일 리스트)


In [ ]:
TABLES_DIR.mkdir(parents=True, exist_ok=True)
FIG_DIR.mkdir(parents=True, exist_ok=True)

tables = sorted(TABLES_DIR.glob("*.csv"))
figs = sorted(FIG_DIR.glob("*.png"))

[t.name for t in tables], [f.name for f in figs]

## 4) 성능 지표 확인 (tables/Day4/day4_metrics.csv)
- uni vs multi 비교
- MAPE는 0 근처에서 왜곡될 수 있어 MAE/RMSE 중심으로 해석


In [ ]:
metrics_path = TABLES_DIR / "day4_metrics.csv"
metrics_df = pd.read_csv(metrics_path)
metrics_df

## 5) 예측 그래프 확인 (예측 + 잔차 비교)
- 상단: actual vs uni vs multi
- 하단: |actual-uni| vs |actual-multi|
- 음영: multi가 더 작은 오차를 보인 구간


In [ ]:
from IPython.display import Image, display

p = FIG_DIR / "forecast_FB_Torque_compare.png"
print(p.name)
display(Image(filename=str(p)))

## 6) 그래프/지표 해석 요약
- 실제 토크는 고주파(초 단위) 변동이 커서, Prophet 기본 구조(추세+회귀변수)만으로는 예측선이 평균 레벨 주변에서 완만하게 움직이는 경향
- 그럼에도 multi는 RPM/압력 정보를 활용해 구간별 평균 레벨을 약간 더 보정하여 MAE가 소폭 개선
- 잔차 그래프에서 multi 오차가 더 작은 구간이 반복적으로 나타나, 개선이 특정 구간에 국한되지 않고 일부 구간에서 지속됨을 확인


## Day5 계획 (이상치 검출)
- 예측 잔차(|actual - pred|) 기반으로 큰 오차 구간을 이상치 후보로 정의
- 범위 기반(out-of-range), 급격한 변화(derivative) 기반 규칙과 함께 이상치 사례를 추출/시각화
- 최종 보고서(PDF) 작성
